In [1]:
import numpy as np
import pandas as pd
import email
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from email.parser import BytesParser
from email.policy import default
import csv
import os
import pathlib
import pprint
from os.path import dirname, abspath

In [2]:
d = dirname(dirname(abspath('__file__')))

In [4]:
#location = f"{d}/case study 3/"

In [3]:
emails = {"index":[]}
for path, subdirectories, filelist in os.walk(os.path.join(d,"SpamAssassinMessages")):
    for f in filelist:
        
        label = 0 if "ham" in path.lower() else 1
        with open(os.path.join(path, f), "rb") as fp:
            msg = BytesParser(policy=default).parse(fp)
            with open(os.path.join(path, f), "r",encoding= 'utf-8', errors='ignore') as new_f:
                msg = email.message_from_file(new_f)
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        body = part.get_payload()  # decode
                emails["index"].append({
                    "Subject": msg.get("Subject"),
                    "From": msg.get("from"),
                    "Body": body,
                    "isSpam": label,
                    "FileID": f
                })

df = pd.DataFrame.from_dict(emails["index"])

In [5]:
df.to_csv("spam_train.csv", index=False)

In [ ]:
df = read_csv("spam_train.csv")

In [4]:
df["isSpam"].value_counts()

0    6954
1    2400
Name: isSpam, dtype: int64

In [14]:
df

,Subject,From,Body,isSpam,FileID
0,None,None,   Bud1           \n                     ...,1,.DS_Store
1,Let us find the right mortgage lender for you ...,pamela4701@eudoramail.com,"Dear Homeowner,\n \nInterest Rates are at thei...",1,00249.5f45607c1bffe89f60ba1ec9f878039a
2,"Friend, Copy ANY DVD or Playstation Game with ...",CopyYourDVD <atomica2020@hotmail.com>,"Friend,Now you can copy DVD's and Games\nhttp:...",1,0355.94ebf637e4bd3db8a81c8ce68ecf681d
3,5% Guaranteed for Eight Years,"""IQ - Safe Harbor"" <sh@insiq.us>",Pocket the newest 8 year annuity!\t Pocket th...,1,0395.bb934e8b4c39d5eab38f828a26f760b4
4,Congratulations! You Get a Free Handheld Organ...,Customer Service <greatoffers@sendgreatoffers....,Pocket the newest 8 year annuity!\t Pocket th...,1,0485.9021367278833179285091e5201f5854
...,...,...,...,...,...
9349,RE: [Razor-users] What's wrong with the Razor ...,"""Rose, Bobby"" <brose@med.wayne.edu>","I'm one of the 30,000 but it's not working ver...",0,00609.dd49926ce94a1ea328cce9b62825bc97
9350,Re: W3C approves HTML 4 'emotitags' [...],harley@argote.ch (Robert Harley),Damien Morton quoted:\n>W3C approves HTML 4 'e...,0,00957.e0b56b117f3ec5f85e432a9d2a47801f
9351,Re: Ximian apt repos?,Mark Derricutt <mark@talios.com>,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n> t...",0,01127.841233b48eceb74a825417d8d918abf8
9352,Re: Installing RPM,Matthias Saou <matthias@egwn.net>,"Once upon a time, Manfred wrote :\n\n> I would...",0,01178.5c977dff972cd6eef64d4173b90307f0


In [15]:
df = df[df["FileID"] != ".DS_Store"]

In [17]:
df.reset_index(inplace=True, drop=True)

In [23]:
df

,Subject,From,Body,isSpam,FileID
0,Let us find the right mortgage lender for you ...,pamela4701@eudoramail.com,"Dear Homeowner,\n \nInterest Rates are at thei...",1,00249.5f45607c1bffe89f60ba1ec9f878039a
1,"Friend, Copy ANY DVD or Playstation Game with ...",CopyYourDVD <atomica2020@hotmail.com>,"Friend,Now you can copy DVD's and Games\nhttp:...",1,0355.94ebf637e4bd3db8a81c8ce68ecf681d
2,5% Guaranteed for Eight Years,"""IQ - Safe Harbor"" <sh@insiq.us>",Pocket the newest 8 year annuity!\t Pocket th...,1,0395.bb934e8b4c39d5eab38f828a26f760b4
3,Congratulations! You Get a Free Handheld Organ...,Customer Service <greatoffers@sendgreatoffers....,Pocket the newest 8 year annuity!\t Pocket th...,1,0485.9021367278833179285091e5201f5854
4,PROTECT YOUR INFORMATION AND YOUR COMPUTER,"""don"" <apf@wu-wien.ac.at>",ATTENTION: This is a MUST for ALL Computer Use...,1,00373.ebe8670ac56b04125c25100a36ab0510
...,...,...,...,...,...
9348,RE: [Razor-users] What's wrong with the Razor ...,"""Rose, Bobby"" <brose@med.wayne.edu>","I'm one of the 30,000 but it's not working ver...",0,00609.dd49926ce94a1ea328cce9b62825bc97
9349,Re: W3C approves HTML 4 'emotitags' [...],harley@argote.ch (Robert Harley),Damien Morton quoted:\n>W3C approves HTML 4 'e...,0,00957.e0b56b117f3ec5f85e432a9d2a47801f
9350,Re: Ximian apt repos?,Mark Derricutt <mark@talios.com>,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n> t...",0,01127.841233b48eceb74a825417d8d918abf8
9351,Re: Installing RPM,Matthias Saou <matthias@egwn.net>,"Once upon a time, Manfred wrote :\n\n> I would...",0,01178.5c977dff972cd6eef64d4173b90307f0


In [40]:
df_null = df.loc[df['Subject'].isnull()]

In [41]:
df_null

,Subject,From,Body,isSpam,FileID
431,None,None,mv 00001.7848dde101aa985090474a91ec93fcf0 0000...,1,cmds
1099,None,"""Vincent Chin"" <nukiez@hotmail.com>",Ripped from\n\nhttp://www.hpl.hp.com/personal/...,0,00175.9836fe00dafac45b3ad3f454ac7e8ee3
1194,None,"""Vincent Chin"" <nukiez@hotmail.com>",Ripped from\n\nhttp://www.hpl.hp.com/personal/...,0,0159.bcc0a572f666161e68caf5313590bd3b
1227,None,None,mv 00001.7c7d6921e671bbe18ebb5f893cd9bb35 0000...,0,cmds
2077,None,None,mv 00001.317e78fa8ee2f54cd4890fdc09ba8176 0000...,1,cmds
5040,None,None,mv 00001.7c53336b37003a9286aba55d2945844c 0000...,0,cmds
8187,None,mail <mail@dogma.slashnull.org>,Problem with spamtrap\n/home/yyyy/lib/spamtrap...,0,01279.36814a9e06b97502d59ec36a27a2adcc
8444,None,nobody@sonic.spamtraps.taint.org (Nobody),Problem with spamtrap\nCould not lock /home/yy...,0,01332.67aa87ef51e1160548ee44dcbef73c42
8569,None,None,mv 00001.d4365609129eef855bd5da583c90552b 0000...,0,cmds
8777,None,nobody@sonic.spamtraps.taint.org (Nobody),Problem with spamtrap\nCould not lock /home/yy...,0,01356.8d72d21568fbfdd4aec060fa8826832a


In [43]:
df_notnull = df.loc[df['Subject'].notnull()]

In [46]:
df_notnull["isSpam"].value_counts()

0    6944
1    2397
Name: isSpam, dtype: int64

Build a spam classifier using naive Bayes and clustering. You will have to create your own dataset from the input messages. Be sure to document how you created your dataset.

In [51]:
#remove new lines and such
df_notnull = df_notnull.replace(r'\n',' ', regex=True)
df_notnull = df_notnull.replace(r'\t',' ', regex=True)
df_notnull = df_notnull.replace(r'b\'',' ', regex=True)
df_notnull = df_notnull.replace(r'b\"',' ', regex=True)


In [52]:
pd.set_option('display.max_colwidth', None)